# Úloha č. 2: Test hypotézy

Overte nasledujúcu hypotézu s 95% istotou:

*Pokiaľ bol vinník nehody pod silným vplyvom alkoholu, došlo častejšie k ťažkým zdravotným následkom.*

Využite **Chi-Square** test.

In [ ]:
###
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import chi2

## Načítanie a vyčistenie dát

In [ ]:
# Načítanie dát
df = pd.read_pickle("accidents.pkl.gz")

df

### Štruktúra stĺpcov *p11*, *p13a* a *p13b*

#### *p11*
Predstavuje prítomnosť alkoholu u vinníka nehody. Povolené sú celé čísla z intervalu **<0,9>**.

| Hodnota | Alkohol |
|---|---|
| 0 | nezisťované |
| 1 | áno, < 0.24 ‰ |
| 2 | nie |
| 3 | áno, >= 0.24 & < 0.5 ‰ |
| 4 | pod vplyvom drog |
| 5 | pod vplyvom alkoholu a drog |
| 6 | áno, >= 0.5 & < 0.8 ‰ |
| 7 | áno, >= 0.8 & < 1.0 ‰ |
| 8 | áno, >= 1.0 & < 1.5 ‰ |
| 9 | áno, >= 1.5 ‰ |

Za silný vplyv alkoholu budeme považovať iba stavy, keď ***p11* >= 7 ** a vylúčime nehody pod vplyvom drog (***p11* = 4** a ***p11* = 5**).


#### *p13a*
Predstavuje počet osôb, ktoré boli usmrtené pri nehode.
Hodnota musí byť celé číslo **>= 0**.

#### *p13b*
Predstavuje počet osôb, ktoré boli ťažko zranené pri nehode.
Hodnota musí byť celé číslo **>= 0**.

Z celého datasetu vylúčime nehody pod vplyvom drog. 

Vytvoríme nový DataFrame so sledovanými parametrami (*p11*, *p13a* a *p13b*). 

In [ ]:
df = df.query('p11 >= 0 & p11 <= 9 & p11 != 4 & p11 != 5')

In [ ]:
# Vytvorenie DataFrame iba so stĺpcami, ktoré potrebujem
df_bis = pd.DataFrame({
          'strong_alcohol_influence': (df['p11'] >= 7).astype(int),
          'total_serious_injuries': df['p13a'] + df['p13b']
         }).dropna()  # vyradiť prázdne

df_bis

In [ ]:
ct = pd.crosstab(df_bis['strong_alcohol_influence'], df_bis['total_serious_injuries'])
ct ###

In [ ]:
stat, p, dof, expected = chi2_contingency(ct)
print('dof=%d' % dof)
print(expected)

In [ ]:
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

In [ ]:
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')